In [1]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import datetime
import sklearn.metrics

In [2]:
# load data_set, and calculate the mean and std of dataset

# transform = transforms.Compose([
#     transforms.Resize((256, 256)),  # 缩放到224 * 224
#     transforms.ToTensor()
# ])

# # 0 -> mask  1-> nonmask  2 -> not a person
# train_dataset = ImageFolder('./data/train', transform=transform)

# # calculate mean of imgs in each RGB channel
# imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
# print(imgs.shape)

# imgs.view(3, -1).mean(dim=1)  # mean of (number of imgs) in each channel
# imgs.view(3, -1).std(dim=1) # std of (number of imgs) in each channel

In [3]:
# load dataset, normarlize it
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 缩放到224 * 224
    # transforms.CenterCrop(256)   #中心剪裁后四周padding补充 (后续可以padding)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4990, 0.4567, 0.4188], std=[0.2913, 0.2778, 0.2836]) 
])

# 0 -> mask  1-> nonmask  2 -> not a person
train_dataset = ImageFolder('./data/train', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = ImageFolder('./data/test', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [4]:
# cnn model

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), # in=3x256x256; out=32x256x256
            nn.ReLU(),
            nn.MaxPool2d(2), # out=32x128x128
            nn.Conv2d(32, 16, kernel_size=3, padding=1), # in=32x128x128, out=16x128x128
            nn.ReLU(),
            nn.MaxPool2d(2), # out=16x64x64
            nn.Conv2d(16, 8, kernel_size=3, padding=1), # in=16x64x64, out=8x64x64
            nn.ReLU(),
            nn.MaxPool2d(2) # out=8x32x32
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(8*32*32, 32*32),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(32*32, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(128, 3)
        )
    
    def forward(self, x):
        # conv layer
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)
        
        return x


In [5]:
device = (torch.device('cuda') if torch.cuda.is_available() 
         else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [6]:
# define train_loop function
def train_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_dataloader: # loop over batches in dataset
            # move data to GPU if available
            imgs = imgs.to(device=device)  
            labels = labels.to(device=device)
            
            outputs = model(imgs)  # feed a batch through our model
            
            loss = loss_fn(outputs, labels)  # computes the loss
            
            optimizer.zero_grad()  # getting rid of the gradients from the last round
            
            loss.backward()  # performs backward step, compute the gradients of all parameters
            
            optimizer.step()  # updates the model
            
            loss_train += loss.item() # sums of losses we saw over the epoch
            
        # print the average loss per batch, in epoch%10 == 0 
        if epoch == 1 or epoch % 5 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch, loss_train/len(train_loader)
            ))

In [7]:
model = CNN().to(device=device)  # instantiates cnn model

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list   # number of parameters, and their shapes

(9577499,
 [864,
  32,
  4608,
  16,
  1152,
  8,
  8388608,
  1024,
  1048576,
  1024,
  131072,
  128,
  384,
  3])

In [8]:
# perform training

learning_rate = 0.001
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()  # use cross entropy loss function

# call train_loop() function
train_loop(
    n_epochs = 20,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_dataloader
)
torch.save(model, 'my_model.pkl')

2021-03-24 21:48:01.443303 Epoch 1, Training loss 0.9623443501336234
2021-03-24 21:48:59.546300 Epoch 5, Training loss 0.3456974787371499
2021-03-24 21:50:08.454302 Epoch 10, Training loss 0.051004125623564635
2021-03-24 21:51:16.420300 Epoch 15, Training loss 0.06964753306071673
2021-03-24 21:52:24.587300 Epoch 20, Training loss 0.023670279951849286


In [9]:
# define validate function
def validate(model, train_loader, test_loader):
    model.eval()
    # accuracy on training data and test data
    for name, loader in [("train", train_loader), ("test", test_loader)]:
        correct = 0
        total = 0
        total_predicted = []
        total_labels = []
        
        with torch.no_grad(): # do not want gradients here, as we will not want to update parameters
            for imgs, labels in loader:
                # move data to GPU if available
                imgs = imgs.to(device=device)  
                labels = labels.to(device=device)
                total_labels.append(labels)
                
                # feed input to models
                outputs = model(imgs)  
                
                # gives the index of the highest value as output
                _, predicted = torch.max(outputs, dim=1)  
                total_predicted.append(predicted)
                
                # counts the number of example, total is increased by the batch size
                total += labels.shape[0]  
                
                # comparing the predicted class that had the maximum probability and the ground-truth labels,
                # we first get a Boolean array. Taking the sum gives the number of items in the batch where 
                # the prediction and ground truth agree
                correct += int((predicted == labels).sum()) 
                
        total_predicted = torch.hstack(total_predicted).cpu()
        total_labels = torch.hstack(total_labels).cpu()
        
        print("Accuracy {}: {:.2f}".format(name, correct / total))  
        print(sklearn.metrics.classification_report(total_labels, total_predicted))
        print(sklearn.metrics.confusion_matrix(total_labels, total_predicted))

In [10]:
# measuring accuracy
validate(model, train_dataloader, test_dataloader)

Accuracy train: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       270
           1       1.00      1.00      1.00       270
           2       1.00      1.00      1.00       576

    accuracy                           1.00      1116
   macro avg       1.00      1.00      1.00      1116
weighted avg       1.00      1.00      1.00      1116

[[270   0   0]
 [  0 270   0]
 [  0   0 576]]
Accuracy test: 0.81
              precision    recall  f1-score   support

           0       0.88      0.69      0.77       123
           1       0.84      0.90      0.87       124
           2       0.73      0.86      0.79       102

    accuracy                           0.81       349
   macro avg       0.82      0.82      0.81       349
weighted avg       0.82      0.81      0.81       349

[[ 85  15  23]
 [  4 111   9]
 [  8   6  88]]
